<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200">
</div>

# Hybrid Compute on Quantinuum H-Series

This notebook contains an example of how to create and run hybrid classical-quantum workflows within the Quantinuum stack via `pytket`.

**Note:** This notebook uses existing Wasm examples provided in this folder's subdirectories. To create and run your own Wasm functions, you'll need to set up an environment on your machine to support this workflow. Instructions for how to do this are given in this folder's README.

This is a simple example made of how you can process measurements, using classical logic via Wasm, to implement a quantum loop, or a repeat-until-success (RUS) style circuit, also called an exit-on-failure circuit.

## Set up `QuantinuumBackend` and load in WASM functions


Note the `wasm` module imported from `pytket` as well as several several conditional operators that are options. More information on the conditional operations available can be found in the user manual at [Classical and Conditional Operations](https://tket.quantinuum.com/api-docs/classical.html).

**Contents**
* [Basic Usage](#Basic-Usage)
* [Repetition Code](#Repetition-Code)
* [Summary](#Summary)

## Basic Usage

In [1]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email:  callum.macpherson@quantinuum.com
Enter your Quantinuum password:  ········


Set up the RUS circuit. This involves 2 steps:
1. Set up Wasm File handler. This checks the Wasm file and can list the available functions within the file.
2. Set up the pytket circuit, which calls the Wasm file handler.

In [2]:
import pathlib
from pytket import wasm

rus_dir = pathlib.Path().cwd().joinpath("RUS")
wasm_file = rus_dir.joinpath("rus_wasm.wasm")
wfh = wasm.WasmFileHandler(wasm_file)

print(repr(wfh))

Functions in wasm file with the uid f5265965c9f207c02e626e3fd48183b545f74877a27c62c3fcc2cc68bee5a36a:
function 'init' with 0 i32 parameter(s) and 0 i32 return value(s)
function 'add_count' with 2 i32 parameter(s) and 1 i32 return value(s)



In [3]:
from pytket.circuit import Circuit, Qubit
from pytket.circuit.logic_exp import reg_lt


def build_rus_circuit(n_repetitions: int, cond_execute: int) -> Circuit:
    """
    n_repetitions (int): number of attempts
    cond_execute: condition to execute, i.e. run this until a total of cond_execute |1> measurements is achieved.

    """
    # Create circuit with two qubits
    circuit = Circuit(2, name=f"RUS_limit {n_repetitions} cond {cond_execute}")

    # Add classical registers
    creg0 = circuit.add_c_register("creg0", 1)
    creg1 = circuit.add_c_register("creg1", 1)
    cond = circuit.add_c_register("cond", 32)
    count = circuit.add_c_register("count", 32)

    # Set cond to 0
    circuit.add_c_setreg(0, cond)

    # Loops
    for loop_iter in range(1, n_repetitions + 1):

        circuit.H(0, condition=reg_lt(cond, cond_execute))
        circuit.CX(0, 1, condition=reg_lt(cond, cond_execute))
        circuit.Measure(Qubit(1), creg1[0], condition=reg_lt(cond, cond_execute))

        # Add wasm call with the parameters creg1 and count, writing the result to cond
        # The function "add_count" is used from the Wasm file
        circuit.add_wasm_to_reg(
            "add_count",
            wfh,
            [creg1, count],
            [cond],
            condition=reg_lt(cond, cond_execute),
        )

        circuit.add_c_setreg(loop_iter, count, condition=reg_lt(cond, cond_execute))
        circuit.Reset(0, condition=reg_lt(cond, cond_execute))

    circuit.Measure(Qubit(0), creg0[0])
    circuit.Measure(Qubit(1), creg1[0])

    return circuit

In [4]:
from pytket.circuit.display import render_circuit_jupyter

circuit = build_rus_circuit(5, 3)

render_circuit_jupyter(circuit)

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [user manual](https://tket.quantinuum.com/user-manual/) for more information on all the options that are available.

Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimisation to perform during compilation, check `pytket-quantinuum` documentation for more information, specifically the [default compilation](https://tket.quantinuum.com/extensions/pytket-quantinuum/#default-compilation) section.

In [5]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=0)

render_circuit_jupyter(compiled_circuit)

Check the cost (in HQCs) of the experiment before running.

In [6]:
n_shots = 1000
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC")

32.0

Note the `wasm_file_handler` input in the `process_circuit` function. This ensures the Wasm functions are passed onto the Quantinuum API.

In [7]:
handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, wasm_file_handler=wfh
)
print(handle)


status = backend.circuit_status(handle)
print(status)


result = backend.get_result(handle)

('5e293c0e388b4dc2948d6588302ac8b8', 'null', 66, '[["cond", 0], ["cond", 1], ["cond", 2], ["cond", 3], ["cond", 4], ["cond", 5], ["cond", 6], ["cond", 7], ["cond", 8], ["cond", 9], ["cond", 10], ["cond", 11], ["cond", 12], ["cond", 13], ["cond", 14], ["cond", 15], ["cond", 16], ["cond", 17], ["cond", 18], ["cond", 19], ["cond", 20], ["cond", 21], ["cond", 22], ["cond", 23], ["cond", 24], ["cond", 25], ["cond", 26], ["cond", 27], ["cond", 28], ["cond", 29], ["cond", 30], ["cond", 31], ["count", 0], ["count", 1], ["count", 2], ["count", 3], ["count", 4], ["count", 5], ["count", 6], ["count", 7], ["count", 8], ["count", 9], ["count", 10], ["count", 11], ["count", 12], ["count", 13], ["count", 14], ["count", 15], ["count", 16], ["count", 17], ["count", 18], ["count", 19], ["count", 20], ["count", 21], ["count", 22], ["count", 23], ["count", 24], ["count", 25], ["count", 26], ["count", 27], ["count", 28], ["count", 29], ["count", 30], ["count", 31], ["creg0", 0], ["creg1", 0]]')
CircuitStat

In [8]:
import json

with open(compiled_circuit.name + ".json", "w", encoding="utf-8") as file:
    json.dump(result.to_dict(), file)

Define a function to analyze the counts of measurements on the `creg1` register. Once we have the counts and number of RUS attempts in hand we can construct a plot for our experiment results.

In [9]:
from pytket.backends.backendresult import BackendResult
from pytket.unit_id import Bit


def analyse_creg1_counts_from_rus_result(
    results_dict: dict[str, BackendResult],
) -> dict[tuple[int, int], int]:
    """Given a dictionary of circuit labels and the corresponding BackendResult objects extract the
    parameters used to construct the circuit as well as the counts on the creg1 register

    In the returned dictionary the key is a tuple of the circuit parameters and the values are
      the counts obtained from the experiment runs.
    """

    creg1_0_cnts = {}
    for circuit_name, backendresult in results_dict.items():

        # extract the integer values of the rus_limit and cond from the circuit name string
        limit, cond = tuple([int(k) for k in circuit_name.split() if k.isdigit()])

        # retrieve the counts of the creg1 qubit for each result
        creg1_counts = backendresult.get_counts(cbits=[Bit("creg1", 0)])

        # store the value of limit and cond with the corresponding counts
        creg1_0_cnts[(limit, cond)] = creg1_counts[(0,)]

    return creg1_0_cnts